In [1]:
from data_utils import read_all_candles, filter_assets
import pandas as pd
from deepdow.utils import raw_to_Xy
from deepdow.benchmarks import OneOverN
from deepdow.data import InRAMDataset, RigidDataLoader
from deepdow.experiments import Run
from deepdow.losses import MaximumDrawdown, SharpeRatio, StandardDeviation, RiskParity
from deepdow.nn import LinearNet, BachelierNet
from deepdow.nn import KeynesNet
from deepdow.nn import ThorpNet
from deepdow.layers import NCO, Cov2Corr, Resample, CovarianceMatrix, AverageCollapse, WeightNorm
from deepdow.benchmarks import Benchmark
import matplotlib.pyplot as plt
import numpy as np
import torch
from deepdow.callbacks import EarlyStoppingCallback
from submission_utils import general_weights_fixer, get_submission_markowitz, test_submission
from sklearn.model_selection import train_test_split
from opt_nets import ResampleNetwork
import os

In [2]:
candles = read_all_candles()
candles2 = read_all_candles("darwins_complete_2803")

Getting candles data for Darwins...: 100%|██████████| 96/96 [00:02<00:00, 34.40it/s]


In [3]:
#candles = {k:v for k,v in candles.items() if k not in assets_0}
#candles2 = {k:v for k,v in candles2.items() if k not in assets_0}

In [4]:
df = pd.concat(candles.values(),keys= candles.keys(),axis=1)

In [5]:
df2 = pd.concat(candles2.values(), keys=candles.keys(), axis=1)

In [6]:
df2 = df2.loc[df2.index >= "2020-06-18"]

In [101]:
df = pd.concat([df, df2])

In [102]:
df = df.loc[df.index >= "2020-03-01"]
# df = df.loc[df.index < "2020-12-25"]


In [6]:
# df = df.loc[df.index < "2020-12-25"]
df = df2.loc[(df2.index >= "2020-08-01") & (df2.index <= "2020-12-25")]

In [7]:
raw_df = df.copy()

In [8]:
n_timesteps = len(raw_df)  # 20
n_channels = len(raw_df.columns.levels[1])  # 2
n_assets = len(raw_df.columns.levels[0])  # 2


In [9]:
n_timesteps, n_channels, n_assets

(3505, 4, 96)

In [10]:
lookback, gap, horizon = 24, 1, 24

X, timestamps, y, asset_names, indicators = raw_to_Xy(raw_df,
                                                      lookback=lookback,
                                                      gap=gap,
                                                      freq="B",
                                                      horizon=horizon)

In [20]:
X.shape

(56, 4, 24, 94)

In [11]:

dataset = InRAMDataset(X, y, timestamps=timestamps, asset_names=asset_names)

X_sample, y_sample, timestamp_sample, asset_names = dataset[0]

In [12]:
ids_tr, ids_val = train_test_split(range(len(dataset)), test_size=0.20, random_state=42)

In [13]:
train_dataloader = RigidDataLoader(dataset, indices=ids_tr, batch_size=16) # list(range(sp_))
val_dataloaders = {'val': RigidDataLoader(dataset, indices=ids_val, batch_size=16)} # list(range(sp_, len(dataset)))

In [14]:
callbacks = [EarlyStoppingCallback("val", "loss", patience=3)]

In [121]:
network2 = LinearNet(n_channels, lookback, n_assets)
loss = SharpeRatio(returns_channel=0)
run2 = Run(network2,
          loss,
          train_dataloader,
          val_dataloaders=val_dataloaders,
          metrics=metrics,
          benchmarks=benchmarks,
          device = torch.device("cuda:0"),
          optimizer=torch.optim.Adam(network.parameters(), lr=0.01), callbacks=callbacks)

In [122]:
history2 = run2.launch(n_epochs=150)

Epoch 0:  67%|██████▋   | 4/6 [00:00<00:00, 24.93it/s, loss=-0.39589, drawdown=0.00291, sharpe=-0.39589]

model   metric    epoch  dataloader
1overN  drawdown  -1     val           0.002
        loss      -1     val          -0.453
        sharpe    -1     val          -0.453


Epoch 11: 100%|██████████| 6/6 [00:00<00:00, 30.55it/s, loss=-0.40252, drawdown=0.00288, sharpe=-0.40252, val_drawdown=0.00220, val_loss=-0.45228, val_sharpe=-0.45228]


Training interrupted
Training stopped early because there was no improvement in val_loss for 3 epochs


In [21]:
max_weight = 0.20
force_symmetric = True
network = ThorpNet(n_assets-2, max_weight=max_weight, force_symmetric=force_symmetric)

In [22]:
#loss = MaximumDrawdown(returns_channel=0) #SharpeRatio(returns_channel=0)
#loss = StandardDeviation()
loss = SharpeRatio(returns_channel=0)
benchmarks = {'1overN': OneOverN()}
metrics = {'drawdown': MaximumDrawdown(returns_channel=0), "sharpe": SharpeRatio(returns_channel=0)}
callbacks = [EarlyStoppingCallback("val", "loss", patience=3)]
run = Run(network,
          loss,
          train_dataloader,
          val_dataloaders=val_dataloaders,
          metrics=metrics,
          benchmarks=benchmarks,
          device = torch.device("cuda:0"),
          optimizer=torch.optim.Adam(network.parameters(), lr=0.001), callbacks=callbacks)

In [23]:
history = run.launch(n_epochs=150)


Epoch 0:   0%|          | 0/3 [00:00<?, ?it/s]

model   metric    epoch  dataloader
1overN  drawdown  -1     val           0.003
        loss      -1     val          -0.066
        sharpe    -1     val          -0.066



Epoch 0: 100%|██████████| 3/3 [00:03<00:00,  1.09s/it, loss=-0.21239, drawdown=0.00264, sharpe=-0.21239, val_drawdown=0.00133, val_loss=-0.35916, val_sharpe=-0.35916]

Epoch 1: 100%|██████████| 3/3 [00:03<00:00,  1.03s/it, loss=-0.51463, drawdown=0.00100, sharpe=-0.51463, val_drawdown=0.00085, val_loss=-0.55515, val_sharpe=-0.55515]

Epoch 2: 100%|██████████| 3/3 [00:03<00:00,  1.08s/it, loss=-0.65993, drawdown=0.00073, sharpe=-0.65993, val_drawdown=0.00072, val_loss=-0.65289, val_sharpe=-0.65289]

Epoch 3: 100%|██████████| 3/3 [00:03<00:00,  1.17s/it, loss=-0.76221, drawdown=0.00062, sharpe=-0.76221, val_drawdown=0.00066, val_loss=-0.73967, val_sharpe=-0.73967]

Epoch 4: 100%|██████████| 3/3 [00:03<00:00,  1.11s/it, loss=-0.85978, drawdown=0.00054, sharpe=-0.85978, val_drawdown=0.00063, val_loss=-0.80900, val_sharpe=-0.80900]

Epoch 5: 100%|██████████| 3/3 [00:03<00:00,  1.22s/it, loss=-0.93559, drawdown=0.00052, sharpe=-0.93559, val_drawdown=0.00062, val_loss=-0.85742, val_sharpe=-0

Epoch 8: 100%|██████████| 3/3 [00:02<00:00,  1.02it/s, loss=-1.09129, drawdown=0.00049, sharpe=-1.09129, val_drawdown=0.00060, val_loss=-0.92595, val_sharpe=-0.92595]

Epoch 9: 100%|██████████| 3/3 [00:03<00:00,  1.14s/it, loss=-1.07584, drawdown=0.00051, sharpe=-1.07584, val_drawdown=0.00063, val_loss=-0.93543, val_sharpe=-0.93543]

Epoch 10: 100%|██████████| 3/3 [00:03<00:00,  1.14s/it, loss=-1.11693, drawdown=0.00050, sharpe=-1.11693, val_drawdown=0.00064, val_loss=-0.94797, val_sharpe=-0.94797]

Epoch 11: 100%|██████████| 3/3 [00:03<00:00,  1.12s/it, loss=-1.11618, drawdown=0.00052, sharpe=-1.11618, val_drawdown=0.00065, val_loss=-0.95608, val_sharpe=-0.95608]

Epoch 12: 100%|██████████| 3/3 [00:03<00:00,  1.06s/it, loss=-1.14067, drawdown=0.00052, sharpe=-1.14067, val_drawdown=0.00066, val_loss=-0.96292, val_sharpe=-0.96292]

Epoch 13: 100%|██████████| 3/3 [00:03<00:00,  1.08s/it, loss=-1.14269, drawdown=0.00053, sharpe=-1.14269, val_drawdown=0.00066, val_loss=-0.97186, val_sharpe

Epoch 16: 100%|██████████| 3/3 [00:03<00:00,  1.10s/it, loss=-1.19909, drawdown=0.00051, sharpe=-1.19909, val_drawdown=0.00066, val_loss=-0.98704, val_sharpe=-0.98704]

Epoch 17: 100%|██████████| 3/3 [00:03<00:00,  1.05s/it, loss=-1.18657, drawdown=0.00055, sharpe=-1.18657, val_drawdown=0.00068, val_loss=-0.98916, val_sharpe=-0.98916]

Epoch 18: 100%|██████████| 3/3 [00:03<00:00,  1.31s/it, loss=-1.18687, drawdown=0.00058, sharpe=-1.18687, val_drawdown=0.00070, val_loss=-0.98949, val_sharpe=-0.98949]

Epoch 19: 100%|██████████| 3/3 [00:03<00:00,  1.13s/it, loss=-1.15740, drawdown=0.00061, sharpe=-1.15740, val_drawdown=0.00071, val_loss=-0.98873, val_sharpe=-0.98873]

Epoch 20: 100%|██████████| 3/3 [00:02<00:00,  1.00it/s, loss=-1.17260, drawdown=0.00061, sharpe=-1.17260, val_drawdown=0.00070, val_loss=-0.99393, val_sharpe=-0.99393]

Epoch 21: 100%|██████████| 3/3 [00:03<00:00,  1.14s/it, loss=-1.20102, drawdown=0.00058, sharpe=-1.20102, val_drawdown=0.00068, val_loss=-0.99801, val_shar

Epoch 24: 100%|██████████| 3/3 [00:03<00:00,  1.09s/it, loss=-1.17179, drawdown=0.00057, sharpe=-1.17179, val_drawdown=0.00067, val_loss=-1.00317, val_sharpe=-1.00317]

Epoch 25: 100%|██████████| 3/3 [00:03<00:00,  1.11s/it, loss=-1.19420, drawdown=0.00056, sharpe=-1.19420, val_drawdown=0.00068, val_loss=-1.00364, val_sharpe=-1.00364]

Epoch 26: 100%|██████████| 3/3 [00:03<00:00,  1.13s/it, loss=-1.19022, drawdown=0.00058, sharpe=-1.19022, val_drawdown=0.00068, val_loss=-1.00631, val_sharpe=-1.00631]

Epoch 27: 100%|██████████| 3/3 [00:03<00:00,  1.23s/it, loss=-1.18524, drawdown=0.00059, sharpe=-1.18524, val_drawdown=0.00068, val_loss=-1.00882, val_sharpe=-1.00882]

Epoch 28: 100%|██████████| 3/3 [00:03<00:00,  1.17s/it, loss=-1.19715, drawdown=0.00060, sharpe=-1.19715, val_drawdown=0.00069, val_loss=-1.01074, val_sharpe=-1.01074]

Epoch 29: 100%|██████████| 3/3 [00:03<00:00,  1.25s/it, loss=-1.20380, drawdown=0.00060, sharpe=-1.20380, val_drawdown=0.00069, val_loss=-1.00884, val_shar

Training interrupted
Training stopped early because there was no improvement in val_loss for 3 epochs


In [24]:
w_pred = run.models["main"].to("cpu")(torch.ones(1, n_channels, lookback, n_assets).cpu()).cpu().detach().numpy().squeeze()

In [25]:
def save_network(experiment_name, run, direc="models_2403"):
    os.makedirs(direc, exist_ok=True)
    network = run.models["main"]
    torch.save(network.state_dict(), f"{direc}/{experiment_name}.pt")

In [26]:
#save_network("mejor_modelo_2903_sharpe_838", run, direc="models_2903")

In [27]:
w_pred

array([ 5.2405827e-12,  9.6017867e-03,  4.8034541e-12,  4.6504688e-12,
       -1.3635857e-12, -7.6696895e-12, -1.6863720e-12,  8.7186797e-03,
        2.5628596e-12,  6.3605119e-12,  2.2893580e-02, -1.0016541e-12,
       -1.3035621e-12,  8.5644247e-13, -1.5659863e-12, -2.3865100e-12,
        7.7200405e-02,  5.1247027e-02,  3.9329212e-02, -2.4057171e-12,
        4.6919982e-13, -1.0947570e-12, -4.7403240e-12,  9.3772537e-13,
       -1.4555120e-12,  1.1043851e-12, -2.3711590e-12,  3.1727277e-02,
        3.0249739e-02,  3.1934358e-02,  2.1053090e-12,  8.9612128e-03,
        4.7505125e-02,  2.5842829e-02,  1.7581703e-02, -1.4261097e-11,
        1.7299064e-12, -4.2742079e-13, -2.4209846e-12,  1.5571607e-12,
        1.1687405e-02,  3.2551929e-02,  1.8870531e-02,  1.0446851e-12,
        4.4903900e-03, -1.1559237e-12,  6.6026273e-12,  2.1247587e-12,
        3.3076510e-02, -1.1307538e-12,  3.1681770e-13,  1.2979887e-12,
        3.4239169e-02,  7.7168709e-03,  2.6082539e-12, -1.0102484e-12,
      

In [28]:
w_pred = general_weights_fixer(w_pred)

In [29]:
#w_pred = w_pred / w_pred.sum()
w_pred = w_pred / sum(w_pred)

In [30]:
dict_weights = {asset:weight for asset,weight in zip(asset_names, w_pred)}

In [31]:
submission = get_submission_markowitz(dict_weights, asset_names)

In [32]:
try:
    test_submission(submission)
except:
    submission = submission.div(submission.sum(axis=1), axis=0)
    test_submission(submission)

In [33]:
submission.to_csv("submission_3103_2.csv", header=True, index=True)

In [34]:
a = pd.read_csv("submission_3103_2.csv", parse_dates=True, index_col="eod_ts")

In [35]:
test_submission(a)

AssertionError: 

In [86]:
# network.load_state_dict(torch.load("models_2903/mejor_modelo_2903_sharpe_838.pt"))

In [26]:
wpreds = network.to("cpu")(torch.ones(1, n_channels, lookback, n_assets).cpu()).cpu().detach().numpy().squeeze()

In [27]:
wpreds

array([ 9.54911844e-12,  6.98230059e-12,  9.03741318e-12,  1.14767822e-02,
        7.75437093e-12,  2.90566105e-02,  1.17503420e-11,  3.22579940e-12,
        1.57612469e-03,  7.84333622e-12,  3.25509541e-12,  9.72465337e-12,
        4.20402905e-12,  2.14313550e-12,  1.19706805e-12, -5.34497204e-14,
        3.48056369e-02,  1.07512455e-02,  3.43189713e-12,  5.64581016e-03,
        3.33843902e-02,  6.74350636e-12,  1.08687321e-11,  7.40081867e-12,
        7.18111941e-12,  2.65190994e-12, -1.23679131e-11,  1.41167590e-12,
        5.38148507e-02,  1.97852924e-02,  3.35596800e-02, -1.06110487e-12,
        6.67711021e-03, -4.99301934e-13,  3.29034925e-02,  3.72512297e-13,
        4.38245907e-02,  2.44144700e-12,  1.27545167e-02,  9.97321496e-12,
        5.91967907e-03,  3.33493203e-03,  4.04355898e-02,  2.55736616e-02,
        7.12052658e-02,  4.06121882e-03, -6.66877621e-12,  3.32778064e-03,
        2.40949057e-02, -5.19079137e-12, -6.62514921e-12,  3.50483271e-12,
        2.99187563e-03,  

In [28]:
assets_0 = [asset for asset, weight in zip(asset_names, wpreds) if weight <= 0]

In [30]:
len(assets_0)

15

In [31]:
assets_0

['EEY',
 'GRU',
 'IDT',
 'JNE',
 'NVL',
 'NYP',
 'OJG',
 'PPT',
 'RWJ',
 'SEH',
 'TKT',
 'WFJ',
 'WWT',
 'YEC',
 'ZTY']